연습문제
목표가 설정
아래 공식을 사용하여 목표가(Target)를 계산합니다.
목표가 = 당일 시가 + (전일 변동폭 * K)
변동폭 = 전일 고가 - 전일 저가
매수 조건
장중 고가가 목표가 이상일 경우 매수.
K값 수익률 분석
다양한 K값(예: 0.1 ~ 1.0)을 사용해 각 K값에 따른 전략 수익률을 계산하세요.
최적 K값 선택
가장 높은 누적 수익률(Cumulative Return)을 기록한 K값을 구하세요.





In [15]:
import pandas as pd 
from pykrx import stock # 9장에 사용법 나옴

In [16]:
df_kosdaq_150 = stock.get_market_ohlcv_by_date('20101001','20241230','000660')
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2010-10-01,22100,22450,21900,22450,8945751,1.354402
2010-10-04,22600,22950,22450,22500,10634669,0.222717
2010-10-05,22350,22750,22000,22750,8459372,1.111111
2010-10-06,23300,23600,23100,23400,15727497,2.857143
2010-10-07,23250,23500,22950,23200,9521740,-0.854701


In [17]:
df_kosdaq_150['변동폭'] = df_kosdaq_150['고가'] - df_kosdaq_150['저가']
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭
날짜,,,,,,,
2010-10-01,22100,22450,21900,22450,8945751,1.354402,550
2010-10-04,22600,22950,22450,22500,10634669,0.222717,500
2010-10-05,22350,22750,22000,22750,8459372,1.111111,750
2010-10-06,23300,23600,23100,23400,15727497,2.857143,500
2010-10-07,23250,23500,22950,23200,9521740,-0.854701,550


In [18]:
df_kosdaq_150['전일변동폭'] = df_kosdaq_150['변동폭'].shift(periods=1) #전일 변동폭이라서 
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭
날짜,,,,,,,,
2010-10-01,22100,22450,21900,22450,8945751,1.354402,550,NaN
2010-10-04,22600,22950,22450,22500,10634669,0.222717,500,550.0
2010-10-05,22350,22750,22000,22750,8459372,1.111111,750,500.0
2010-10-06,23300,23600,23100,23400,15727497,2.857143,500,750.0
2010-10-07,23250,23500,22950,23200,9521740,-0.854701,550,500.0


In [19]:
K = 1/2
df_kosdaq_150['목표가'] = df_kosdaq_150['시가'] + (df_kosdaq_150['전일변동폭'] * K)
df_kosdaq_150.head()

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭,목표가
날짜,,,,,,,,,
2010-10-01,22100,22450,21900,22450,8945751,1.354402,550,NaN,NaN
2010-10-04,22600,22950,22450,22500,10634669,0.222717,500,550.0,22875.0
2010-10-05,22350,22750,22000,22750,8459372,1.111111,750,500.0,22600.0
2010-10-06,23300,23600,23100,23400,15727497,2.857143,500,750.0,23675.0
2010-10-07,23250,23500,22950,23200,9521740,-0.854701,550,500.0,23500.0


In [20]:
conditions = df_kosdaq_150['고가']>= df_kosdaq_150['목표가']  # 매도 조건

In [21]:
import numpy as np
df_kosdaq_150['수익률'] = np.where(conditions, df_kosdaq_150['종가'] / df_kosdaq_150['목표가'], 1)

In [22]:
delta_days = df_kosdaq_150.index[-1] - df_kosdaq_150.index[0]
year = delta_days.days / 365

CAGR_dict = {}
for k in range(1,11):
    df_kosdaq_150['목표가'] = df_kosdaq_150['시가'] + (df_kosdaq_150['전일변동폭'] * (k/10))
    df_kosdaq_150['수익률'] = np.where(conditions, df_kosdaq_150['종가'] / df_kosdaq_150['목표가'], 1)
    df_kosdaq_150['누적수익률'] = df_kosdaq_150['수익률'].cumprod()
    CAGR = df_kosdaq_150['누적수익률'].iloc[-1] ** (1/year) -1
    CAGR_dict[f'{(k/10)}'] = CAGR * 100


CAGR_dict

{'0.1': 183.95569534843506,
 '0.2': 117.97615987961652,
 '0.3': 67.45978944205532,
 '0.4': 28.75165281015548,
 '0.5': -0.9319895656322741,
 '0.6': -23.71307978295517,
 '0.7': -41.21041142734197,
 '0.8': -54.659931779675695,
 '0.9': -65.00604193836537,
 '1.0': -72.9709438527998}